In [1]:
import json
from tqdm import tqdm
from IPython.display import clear_output
from database_connect import get_database

In [2]:
db = get_database()

In [6]:
def update_item_cat(item, category):
    query = {'item': str(item).lower()}
    insert = {'item': str(item).lower(), 'category': str(category).lower(), 'reviewed': True}

    if db.item_category_map.find_one(query) is not None:
        result = db.item_category_map.replace_one(query, insert)
    else:
        result = db.item_category_map.insert_one(insert)
    
    return result

### Backfill Process

In [5]:
backfill_document = 'C:/Users/BAB72/OneDrive/Documents/GitHub/recipegen.github.io/scripts/recipe_database/item_category_mapping.json'
with open(backfill_document, 'r') as file:
    backfill_dict = json.loads(file.readline())
    
for item in backfill_dict:
    clear_output()
    print('Item:\t' + item)
    print('Category:\t' + backfill_dict[item])
    accept = input('Accept:\t')
    
    if accept.lower() == 'y':
        update_item_cat(item, backfill_dict[item])
    elif accept.lower() == 'n':
        continue
    else:
        update_item_cat(item, accept)

Item:	pie crust
Category:	Bakery


Accept:	 y


In [7]:
for item in db.item_category_map.find({'category': 'other'}):
    print(item)

{'_id': ObjectId('638f57d8013b2399b4b833ad'), 'item': 'tofu', 'category': 'other'}
{'_id': ObjectId('638f57e8013b2399b4b833b5'), 'item': 'eggs', 'category': 'other'}
{'_id': ObjectId('638f580d013b2399b4b833c8'), 'item': 'wooden skewers', 'category': 'other'}
{'_id': ObjectId('638f5994013b2399b4b8343f'), 'item': 'box', 'category': 'other'}
{'_id': ObjectId('638f64c3013b2399b4b83445'), 'item': 'eggs*', 'category': 'other'}
{'_id': ObjectId('638f65dc013b2399b4b834b9'), 'item': 'oven-ready aluminum trays', 'category': 'other'}
{'_id': ObjectId('638f664a013b2399b4b834de'), 'item': 'silken tofu', 'category': 'other'}


### New Entry Process

In [10]:
item_list = []
for recipe_details in db.recipe_details.find():
    for recipe_item in recipe_details['recipe']:
        item_list.append(recipe_item['item'])
item_set = list(set(item_list))

set()

In [ ]:
for i in range(len(item_set)):
    if db.item_category_map.find({'item': item_set[i]}) is None:
        clear_output()
        print(str(i+1) + '/' + str(len(item_set)))
        print('Item:\t' + item_set[i])
        category = input('Category:\t')

        if category.lower() == 'n':
            continue
        else:
            update_item_cat(item, category)